# Introdução à Probabilidade e Estatística
Neste notebook, vamos explorar alguns dos conceitos que discutimos anteriormente. Muitos conceitos de probabilidade e estatística estão bem representados nas principais bibliotecas para processamento de dados em Python, como `numpy` e `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Variáveis Aleatórias e Distribuições
Vamos começar desenhando uma amostra de 30 valores de uma distribuição uniforme de 0 a 9. Também vamos calcular a média e a variância.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

Para estimar visualmente quantos valores diferentes estão na amostra, podemos plotar o **histograma**:


In [ ]:
plt.hist(sample)
plt.show()

## Analisando Dados Reais

Média e variância são muito importantes ao analisar dados do mundo real. Vamos carregar os dados sobre jogadores de beisebol de [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Estamos usando um pacote chamado [**Pandas**](https://pandas.pydata.org/) aqui para análise de dados. Falaremos mais sobre Pandas e trabalhar com dados em Python mais adiante neste curso.

Vamos calcular os valores médios para idade, altura e peso:


In [ ]:
df[['Age','Height','Weight']].mean()

Agora vamos focar na altura e calcular o desvio padrão e a variância:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

Além da média, faz sentido olhar o valor mediano e os quartis. Eles podem ser visualizados usando um **box plot**:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Também podemos fazer gráficos de caixa de subconjuntos do nosso conjunto de dados, por exemplo, agrupados por função do jogador.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Nota**: Este diagrama sugere que, em média, as alturas dos primeiros base são maiores do que as alturas dos segundos base. Mais adiante, aprenderemos como podemos testar essa hipótese de forma mais formal e como demonstrar que nossos dados são estatisticamente significativos para mostrar isso.

Idade, altura e peso são todas variáveis aleatórias contínuas. O que você acha que é a distribuição delas? Uma boa maneira de descobrir é traçar o histograma dos valores:


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Distribuição Normal

Vamos criar uma amostra artificial de pesos que segue uma distribuição normal com a mesma média e variância dos nossos dados reais:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Como a maioria dos valores na vida real segue uma distribuição normal, não devemos usar um gerador de números aleatórios uniforme para gerar dados amostrais. Aqui está o que acontece se tentarmos gerar pesos com uma distribuição uniforme (gerada por `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Intervalos de Confiança

Vamos agora calcular intervalos de confiança para os pesos e alturas dos jogadores de beisebol. Usaremos o código [desta discussão no stackoverflow](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Teste de Hipóteses

Vamos explorar diferentes papéis em nosso conjunto de dados de jogadores de beisebol:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Vamos testar a hipótese de que os Primeiros Bases são mais altos do que os Segundos Bases. A maneira mais simples de fazer isso é testar os intervalos de confiança:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Podemos ver que os intervalos não se sobrepõem.

Uma maneira estatisticamente mais correta de provar a hipótese é usar um **teste t de Student**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

Os dois valores retornados pela função `ttest_ind` são:
* p-valor pode ser considerado como a probabilidade de duas distribuições terem a mesma média. No nosso caso, é muito baixo, o que significa que há fortes evidências de que os primeiras bases são mais altos.
* t-valor é o valor intermediário da diferença média normalizada que é usado no teste t, e é comparado contra um valor limite para um dado valor de confiança.


## Simulando uma Distribuição Normal com o Teorema do Limite Central

O gerador pseudoaleatório em Python é projetado para nos fornecer uma distribuição uniforme. Se quisermos criar um gerador para distribuição normal, podemos usar o teorema do limite central. Para obter um valor distribuído normalmente, basta calcular a média de uma amostra gerada uniformemente.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Correlação e Evil Baseball Corp

A correlação nos permite encontrar relações entre sequências de dados. No nosso exemplo lúdico, vamos fingir que existe uma corporação de beisebol malvada que paga seus jogadores de acordo com a altura - quanto mais alto o jogador, mais dinheiro ele/ela recebe. Suponha que haja um salário base de $1000, e um bônus adicional de $0 a $100, dependendo da altura. Vamos pegar os jogadores reais da MLB e calcular seus salários imaginários:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Vamos agora calcular a covariância e a correlação dessas sequências. `np.cov` nos dará a chamada **matriz de covariância**, que é uma extensão da covariância para múltiplas variáveis. O elemento $M_{ij}$ da matriz de covariância $M$ é uma correlação entre as variáveis de entrada $X_i$ e $X_j$, e os valores diagonais $M_{ii}$ são a variância de $X_{i}$. Da mesma forma, `np.corrcoef` nos dará a **matriz de correlação**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Uma correlação igual a 1 significa que existe uma **relação linear** forte entre duas variáveis. Podemos visualizar a relação linear ao plotar um valor contra o outro:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Vamos ver o que acontece se a relação não for linear. Suponha que nossa empresa decidiu esconder a óbvia dependência linear entre alturas e salários, e introduziu alguma não linearidade na fórmula, como `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Neste caso, a correlação é um pouco menor, mas ainda é bastante alta. Agora, para tornar a relação ainda menos óbvia, podemos querer adicionar um pouco de aleatoriedade extra adicionando uma variável aleatória ao salário. Vamos ver o que acontece:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Você consegue adivinhar por que os pontos se alinham em linhas verticais assim?

Observamos a correlação entre um conceito artificialmente criado como salário e a variável observada *altura*. Vamos também ver se as duas variáveis observadas, como altura e peso, também se correlacionam:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

Infelizmente, não obtivemos nenhum resultado - apenas alguns valores estranhos `nan`. Isso ocorre porque alguns dos valores em nossa série estão indefinidos, representados como `nan`, o que faz com que o resultado da operação também seja indefinido. Ao olhar para a matriz, podemos ver que `Weight` é a coluna problemática, porque a autocorrelação entre os valores de `Height` foi calculada.

> Este exemplo mostra a importância da **preparação** e **limpeza** dos dados. Sem dados adequados, não podemos calcular nada.

Vamos usar o método `fillna` para preencher os valores ausentes e calcular a correlação: 


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

De fato, existe uma correlação, mas não tão forte quanto em nosso exemplo artificial. Na verdade, se analisarmos o gráfico de dispersão de um valor em relação ao outro, a relação seria muito menos óbvia:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Conclusão

Neste notebook aprendemos como realizar operações básicas em dados para calcular funções estatísticas. Agora sabemos como usar um aparato sólido de matemática e estatística para provar algumas hipóteses, e como calcular intervalos de confiança para variáveis arbitrárias dadas uma amostra de dados.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automáticas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte autoritária. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes do uso desta tradução.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
